In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2, l1



c:\Users\adria\.conda\envs\trash_app\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
path = "proccesed_dataset"

In [3]:
# Define path to your dataset (replace with actual path if needed)
def load_and_split_data(directory, validation_split=0.2, test_split=0.1, seed=123):
    # Load the entire dataset
    full_dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        seed=seed,
        shuffle=True,
        label_mode='categorical',
        image_size=(224, 224),
        batch_size=32) 

    # Calculate the number of batches needed for each split
    total_batches = len(full_dataset)
    val_batches = int(total_batches * validation_split)
    test_batches = int(total_batches * test_split)
    train_batches = total_batches - val_batches - test_batches

    # Split the dataset into train, validation, and test
    train_dataset = full_dataset.take(train_batches)
    test_dataset = full_dataset.skip(train_batches).take(test_batches)
    validation_dataset = full_dataset.skip(train_batches + test_batches)

    return train_dataset, validation_dataset, test_dataset


In [4]:
base_dir = 'proccesed_dataset'
train_ds, val_ds, test_ds = load_and_split_data(base_dir)

Found 8303 files belonging to 6 classes.


In [5]:
def make_model(input_shape, num_classes):
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2)
    ])
    
    base_model = EfficientNetB7(input_shape=input_shape,
                            include_top=False,
                            weights='imagenet')
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False
    
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = base_model(inputs, training=True)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    
    # Increased L2 regularization for weight decay effect
    x = Dense(2048, activation='relu', 
              kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.7)(x)
    
    x = Dense(1024, activation='relu', 
              kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.6)(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

In [6]:
model = make_model(input_shape=(224, 224, 3), num_classes=6)
model.compile(
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=6e-5
    ),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
class AccuracyThreshold(Callback):
    def __init__(self, threshold=0.85):
        super(AccuracyThreshold, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get("val_accuracy")
        accuracy = logs.get("accuracy")
        if val_accuracy is not None:
            if val_accuracy >= self.threshold:
                print(f"\nReached {self.threshold * 100}% val accuracy. Stopping training...")
                self.model.stop_training = True
            elif accuracy >= 0.95:
                print(f"\nReached 95% accuracy. Stopping training...")
                self.model.stop_training = True

# Instantiate the custom callback with a threshold of 85% (0.85)
accuracy_threshold_callback = AccuracyThreshold(threshold=0.90)

# Early Stopping and Model Checkpoint
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,  # More patience
    restore_best_weights=True,
    min_delta=0.001  # Minimum change to qualify as an improvement
)



In [8]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[accuracy_threshold_callback,early_stopping] 
)

Epoch 1/50
182/182 [==============================] - 552s 3s/step - loss: 37.1595 - accuracy: 0.5737 - val_loss: 35.5565 - val_accuracy: 0.8221
Epoch 2/50
182/182 [==============================] - 534s 3s/step - loss: 34.9331 - accuracy: 0.7641 - val_loss: 33.7193 - val_accuracy: 0.8597
Epoch 3/50
182/182 [==============================] - 528s 3s/step - loss: 33.0659 - accuracy: 0.8110 - val_loss: 31.9319 - val_accuracy: 0.8767
Epoch 4/50
182/182 [==============================] - 529s 3s/step - loss: 31.1927 - accuracy: 0.8388 - val_loss: 30.1299 - val_accuracy: 0.8871
Epoch 5/50
182/182 [==============================] - 516s 3s/step - loss: 29.3278 - accuracy: 0.8599 - val_loss: 28.3268 - val_accuracy: 0.8919
Epoch 6/50
182/182 [==============================] - 526s 3s/step - loss: 27.4895 - accuracy: 0.8743 - val_loss: 26.5533 - val_accuracy: 0.8962
Epoch 7/50
182/182 [==============================] - ETA: 0s - loss: 25.6696 - accuracy: 0.8934
Reached 90.0% val accuracy. Stopp

In [9]:
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


26/26 [==============================] - 56s 2s/step - loss: 24.7946 - accuracy: 0.9171
Test accuracy: 0.917067289352417
Test loss: 24.794639587402344


In [10]:
for layer in model.layers:
    for attr, value in layer.__dict__.items():
        if isinstance(value, tf.Tensor):
            print(f"Layer {layer.name} has a non-serializable attribute {attr} with value {value}.")



In [11]:
model.save('model/EfficientNetB7')



INFO:tensorflow:Assets written to: model/EfficientNetB7\assets


INFO:tensorflow:Assets written to: model/EfficientNetB7\assets


In [12]:
import pandas as pd

In [13]:
# Convert to DataFrame
history_df = pd.DataFrame(history.history)

# Save to CSV
history_df.to_csv('model_history.csv', index=False)